In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：超参数调整图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_image_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库为自定义图像分类模型进行超参数调整。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/cifar10)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/overview)。您将使用的数据集版本已内置在 TensorFlow 中。训练好的模型可以预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在这本笔记本中，您将学习如何使用 Vertex 客户端库在 Docker 容器中的 Python 脚本中创建一个自定义图像分类模型的超参数调整作业。您还可以使用 `gcloud` 命令行工具或在线使用 Google Cloud 控制台来调整模型的超参数。


执行的步骤包括：

- 创建一个 Vertex 超参数调整作业来训练一个自定义模型。
- 调整自定义模型。
- 评估研究结果。

### 成本

此教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了 Vertex 客户端库和 Google *cloud-storage*，您需要重新启动笔记本的内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*确保如果有这个选项的话，你正在 GPU 运行时中运行这个笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置你的谷歌云项目

**下面的步骤是必需的，无论你的笔记本环境如何。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个账户时，你会获得一个 $300 的免费信用，用于您的计算/存储费用。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用顶点 API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入你的项目 ID。然后运行该单元格，以确保 Cloud SDK 在这个笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是Vertex支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多地区存储桶来进行Vertex的训练。并非所有地区均支持所有Vertex服务。有关每个地区的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在一个实时教程会话中，可能会使用一个共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源名称。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户访问项目的权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，然后选中**Vertex管理员**。在过滤框中键入“Storage Object Admin”，然后选中**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

将服务帐户密钥的路径输入到下面单元格中作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪个笔记本环境，都需要执行以下步骤。**

使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 将从这个包中运行代码。在本教程中，Vertex 还会将训练作业产生的训练模型保存在同一个存储桶中。然后您可以基于这个输出创建一个`Endpoint`资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括您组织外的那些项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才运行以下命令以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage存储桶中的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`: 用于数据集、模型、作业、管道和端点服务的顶点API服务端点。
- `PARENT`: 用于数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）映像

接下来，我们将设置用于训练的Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`将是版本2.1，`1-15`将是版本1.15。以下列表显示了一些预构建的可用映像：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 机器类型

接下来，设置用于训练的机器类型。

- 设置变量`TRAIN_COMPUTE`来配置用于训练的虚拟机的计算资源。
   - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB的内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB的内存
     - `n1-highcpu`：每个虚拟CPU 0.9GB的内存
  - `虚拟CPU数`：\[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下内容不支持训练：*

   - `standard`：2个虚拟CPU
   - `highcpu`：2、4和8个虚拟CPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

# 教程

现在你已准备好开始创建自己的超参数调整和训练一个定制图像分类器。

## 设置客户端

Vertex 客户端库以客户端/服务器模型工作。在您的一侧（Python 脚本），您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程的不同步骤使用不同的客户端。因此，请提前设置它们。

- 为“Model”资源设置模型服务。
- 为超参数调优设置作业服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()

for client in clients.items():
    print(client)

调整模型 - 你好世界

有两种方法可以使用容器映像来调整超参数并训练自定义模型：

- **使用Google Cloud预构建的容器**。如果使用预构建的容器，您还将指定要安装到容器映像中的Python软件包。这个Python软件包包含了用于超参数调整和训练自定义模型的代码。

- **使用自己的自定义容器映像**。如果使用自己的容器，容器需要包含用于超参数调整和训练自定义模型的代码。

准备好你的超参数调整作业规范

现在你的客户准备好了，你的第一步是为你的超参数调整作业创建一个作业规范。 作业规范将包括以下内容：

- `trial_job_spec`：自定义作业的规范。
  - `worker_pool_spec`：用于超参数调整的机器类型和数量的规范（单一或分布式）
  - `python_package_spec`：要与预构建容器一起安装的Python包的规范。

- `study_spec`：对调整内容的规范。
  - `parameters`：这是您将为自定义训练作业进行调整的超参数的规范。 它将包含一个列表
  - `metrics`：这是如何评估每次调整试验结果的规范。

### 准备您的机器规格

现在为您定制的超参数调整工作定义机器规格。这将告诉 Vertex 需要为超参数调整提供哪种类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型（如果有的话）。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则使用 GPU；否则您将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义超参数调整作业定义磁盘规格。这告诉 Vertex 每个机器实例为超参数调整提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您需要为自定义超参数调整作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`: 该机器类型要预留的实例数。
- `machine_spec`: 硬件规范。
- `disk_spec`:（可选）磁盘存储规范。

- `python_package`: 要在VM实例上安装的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包的规范：

- `executor_image_spec`: 这是为您的自定义超参数调整作业配置的docker镜像。

- `package_uris`: 这是您的Python训练包的位置（URI）列表，要安装在预留实例上。这些位置需要在云存储存储桶中。它们可以是单个Python文件，也可以是整个包的zip（归档）。在后一种情况下，作业服务将解压（解档）内容到docker镜像中。

- `python_module`: 用于运行自定义超参数调整作业的Python模块（脚本）。在这个例子中，您将调用`trainer.task.py` -- 请注意，不需要附加`.py`后缀。

- `args`: 要传递给相应Python模块的命令行参数。在这个例子中，您将设置：
  - `"--model-dir=" + MODEL_DIR` : 存储模型构件的云存储位置。有两种方法告诉超参数调整脚本在哪里保存模型构件：
      - 直接：将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`）。
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型构件的位置。
  - `"--epochs=" + EPOCHS`: 训练时的epoch数。
  - `"--steps=" + STEPS`: 每个epoch的步数（批次数）。
  - `"--distribute=" + TRAIN_STRATEGY"` : 用于单一或分布式超参数调整的超参数调整分布策略。
     - `"single"`: 单个设备。
     - `"mirror"`: 单个计算实例上的所有GPU设备。
     - `"multi"`: 所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 创建研究规范

让我们从一个简单的研究开始。您只会使用一个参数 -- *学习率*。由于只有一个参数，进行随机搜索并没有太多意义。相反，我们将在一系列值上进行网格搜索。

- `度量`:
 - `度量ID`: 在这个例子中，要报告回来的客观度量是`'val_accuracy'`。
 - `目标`: 在这个例子中，超参数调整服务将评估试验以最大化客观度量的值。
- `参数`: 调整超参数的规范。
 - `参数ID`: 将作为命令行参数传递给Python软件包的超参数的名称。
  - `缩放类型`: 缩放类型决定了超参数调整服务在搜索空间中搜索时使用的分辨率。
   - `UNIT_LINEAR_SCALE`: 在搜索空间中分辨率相同。
   - `UNIT_LOG_SCALE`: 靠近搜索空间底部的值距离更远。
   - `UNIT_REVERSE_LOG_SCALE`: 靠近搜索空间顶部的值距离更远。
 - **搜索空间**: 在这里，您将指定用于选择调整超参数的值的搜索空间。
   - `整数值规范`: 指定在`min_value`和`max_value`之间的整数范围的值。
   - `双精度值规范`: 指定在`min_value`和`max_value`之间的连续范围的值。
   - `离散值规范`: 指定一个值列表。
- `算法`: 每次试验选择超参数值的搜索方法:
 - `GRID_SEARCH`: 组合搜索 -- 在这个例子中使用。
 - `RANDOM_SEARCH`: 随机搜索。

In [ ]:
study_spec = {
    "metrics": [
        {
            "metric_id": "val_accuracy",
            "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE,
        }
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        }
    ],
    "algorithm": aip.StudySpec.Algorithm.GRID_SEARCH,
}

### 组装一个超参数调整作业规范

现在，组装自定义超参数调整规范的完整描述：

- `display_name`：您为此自定义超参数调整作业分配的可读名称。
- `trial_job_spec`：自定义超参数调整作业的规范。
- `study_spec`：要调整的规范。
- `max_trial_count`：调整试验的最大数量。
- `parallel_trial_count`：要并行尝试的试验数量；否则，它们将顺序进行。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

### 检查超参数调整包

#### 包布局

在开始进行超参数调整之前，您将查看一个用于自定义超参数调整作业的Python包是如何组装的。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的指令。

文件`trainer/task.py`是执行自定义超参数调整作业的Python脚本。*请注意*，当我们在工作线程池规范中提及它时，我们用一个点(`trainer.task`)替换目录斜杠，并删除文件后缀(`.py`)。

#### 包组装

在接下来的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration hyperparameter tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您可以编写超参数调整脚本task.py的内容。我不会详细介绍，只是供您查阅。总结如下：

- 作为命令行参数传递试验的超参数值（`parser.add_argument('--lr',...)`）
- 模拟训练循环，在每个循环（轮次）中，将变量`accuracy`设置为循环迭代次数乘以学习率。
- 使用`report_hyperparameter_tuning_metric()`将计算的目标指标`accuracy`反馈给超参数调整服务。

In [ ]:
%%writefile custom/trainer/task.py
# HP Tuning hello world example

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import argparse
import os
import sys
import time
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--model-dir',
                    dest='model_dir',
                    default='/tmp/saved_model',
                    type=str,
                    help='Model dir.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Instantiate the HyperTune reporting object
hpt = HyperTune()

for epoch in range(1, args.epochs+1):
    # mimic metric result at the end of an epoch
    acc = args.lr * epoch
    # save the metric result to communicate back to the HPT service
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_accuracy',
        metric_value=acc,
        global_step=epoch)
    print('epoch: {}, accuracy: {}'.format(epoch, acc))
    time.sleep(1)

#### 将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 使用HyperTune报告试验结果

对于每次试验，您的Python脚本需要向超参数调整服务报告您指定为评估试验的标准的客观指标。

在本示例中，您将在研究规范中指定客观指标将被报告为`val_accuracy`。

您可以使用`HyperTune`报告客观指标的数值。这个Python模块用于向超参数调整服务传递关键/值对。为了设置这个报告在您的Python包中，您需要添加以下三个步骤的代码：

1. 导入`HyperTune`模块：`from hypertune import HyperTune()`。
2. 在每个epoch结束时，使用`hpt.report_hyperparameter_tuning_metric()`将客观函数的当前值作为关键/值对写入日志。在这个示例中，参数是：
 - `hyperparameter_metric_tag`：要报告的客观指标的名称。名称必须与研究规范中指定的名称完全相同。
 - `metric_value`：要报告给超参数服务的客观指标的值。
 - `global_step`：epoch迭代的全局步骤，从0开始。

## 超参数调整模型

现在开始在Vertex上调整自定义模型的超参数。使用这个帮助函数 `create_hyperparameter_tuning_job`，它接受以下参数：

-`hpt_job`: 超参数调整作业的规范。

该帮助函数调用作业客户端服务的 `create_hyperparameter_tuning_job` 方法，带有以下参数：

-`parent`: `Dataset`、`Model` 和 `Endpoint` 资源的Vertex位置路径。
-`hyperparameter_tuning_job`: 超参数调整作业的规范。

您将会展示在 `response` 对象中返回的一些字段，最感兴趣的两个字段是：

`response.name`: 分配给此自定义超参数调整作业的Vertex完整标识符。您将保存这个标识符以在后续步骤中使用。

`response.state`: 自定义超参数调整作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获得您创建的超参数调优作业的唯一标识符。

In [ ]:
# The full unique ID for the hyperparameter tuning job
hpt_job_id = response.name
# The short numeric ID for the hyperparameter tuning job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

获取超参数调整作业的信息

接下来，使用这个辅助函数`get_hyperparameter_tuning_job`，它接受以下参数：

- `name`: 超参数调整作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_hyperparameter_tuning_job`方法，参数如下：

- `name`: 超参数调整作业的 Vertex 完全限定标识符。

如果你记得，当你调用`create_hyperparameter_tuning_job`方法时，在`response.name`字段中获得了超参数调整作业的 Vertex 完全限定标识符，并将标识符保存在变量`hpt_job_id`中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

等待调整完成

对上述模型进行超参数调整可能需要20分钟以上的时间。

一旦您的模型调整完成，您可以通过将`end_time`减去`start_time`来计算调整模型所花费的实际时间。

对于您的模型，我们需要知道每个试验保存模型的位置，Python脚本将其保存在您本地的云存储存储桶中的`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

### 请审查研究结果

现在审查试验结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

### 最佳试验

现在看看哪个试验是最好的：

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

## 获取最佳模型

如果您使用了让服务告知调参脚本在哪里保存模型构件的方法（`DIRECT = False`），那么最佳模型的模型构件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

## 调整模型 - CIFAR10

现在您已经了解了使用一个模拟训练模型的Python包进行超参数调整的整体步骤，您将为一个CIFAR10模型的自定义训练工作做一个新的超参数调整工作。

在这个例子中，您需要更改两个部分：

1. 指定CIFAR10自定义超参数调整的Python包。
2. 指定一个针对CIFAR10自定义超参数调整的Python包中使用的超参数的研究规范。

### 创建一个研究规范

在这项研究中，您将使用随机搜索算法优化两个超参数：

- **学习率**：搜索空间是一组离散值。
- **学习率衰减**：搜索空间是1e-6到1e-2之间的连续范围。

目标（目标）是最大化验证准确性。

您将最多运行六次试验。

In [ ]:
study_spec = {
    "metrics": [
        {
            "metric_id": "val_accuracy",
            "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE,
        }
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
        {
            "parameter_id": "decay",
            "double_value_spec": {"min_value": 1e-6, "max_value": 1e-2},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
    ],
    "algorithm": aip.StudySpec.Algorithm.RANDOM_SEARCH,
}

### 汇编超参数调整作业规范

现在汇编自定义超参数调整规范的完整描述：

- `display_name`：您为此自定义超参数调整作业分配的人类可读名称。
- `trial_job_spec`：自定义超参数调整作业的规范。
- `study_spec`：要调整的内容规范。
- `max_trial_count`：最大调整试验次数。
- `parallel_trial_count`：要并行尝试的试验数量；否则，它们将按顺序进行。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

#### Task.py的内容

在下一个单元格中，您可以编写超参数调整脚本task.py的内容。我不会详细介绍，只是为了让您浏览一下。总结如下：

- 解析命令行参数，设置当前试验的超参数。
    - 从命令行获取保存模型文件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 下载并预处理CIFAR10数据集。
- 构建一个CNN模型。
- 在编译模型时使用学习率和衰减超参数值。
- 定义一个名为`HPTCallback`的回调函数，该回调函数在每个epoch结束时获取验证准确率（`on_epoch_end()`）并报告给超参数调整服务使用`hpt.report_hyperparameter_tuning_metric()`。
- 使用`fit()`方法训练模型，并指定一个回调函数，将验证准确率报告回超参数调整服务。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Job for CIFAR10

import tensorflow_datasets as tfds
import tensorflow as tf
from hypertune import HyperTune
import argparse
import os
import sys

# Command Line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()


# Scaling CIFAR-10 data from (0, 255] to (0., 1.]
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


# Download the dataset
datasets = tfds.load(name='cifar10', as_supervised=True)

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = datasets['train'].map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = datasets['test'].map(scale).batch(BATCH_SIZE)

# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr, decay=args.decay),
      metrics=['accuracy'])
  return model


model = build_and_compile_cnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_accuracy',
        metric_value=logs['val_accuracy'],
        global_step=epoch)

# Train the model
model.fit(train_dataset, epochs=5, steps_per_epoch=10, validation_data=test_dataset.take(8),
    callbacks=[HPTCallback()])
model.save(args.model_dir)

把超参数调整脚本存储在您的云存储桶上

接下来，您将超参数调整文件夹打包成压缩的 tar 文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

#### 使用 hypertune 报告试验结果

对于每次试验，你的 Python 脚本需要向超参数调整服务报告你指定的评估标准的客观指标。

在这个示例中，你需要在研究规范中指定客观指标将被报告为 `val_accuracy`。

你可以使用 `HyperTune` 报告客观指标的数值。这个 Python 模块用于向超参数调整服务传递键值对。为了在你的 Python 包中设置这个报告，你需要添加以下三个步骤的代码：

1. 导入 HyperTune 模块：`from hypertune import HyperTune()`。
2. 在每个周期结束时，使用 `hpt.report_hyperparameter_tuning_metric()` 将客观函数的当前值写入日志作为键值对。在这个示例中，参数为：
 - `hyperparameter_metric_tag`：需要报告的客观指标的名称。这个名称必须与研究规范中指定的名称完全相同。
 - `metric_value`：需要向超参数服务报告的客观指标值。
 - `global_step`：周期迭代次数，从 0 开始计数。

## 超参数调整模型

现在开始在 Vertex 上对自定义模型进行超参数调整。使用这个辅助函数 `create_hyperparameter_tuning_job`，它接收以下参数：

- `hpt_job`：用于超参数调整作业的规范。

辅助函数调用作业客户端服务的 `create_hyperparameter_tuning_job` 方法，传入以下参数：

- `parent`：指向 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置路径。
- `hyperparameter_tuning_job`：用于超参数调整作业的规范。

您将显示 `response` 对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义超参数调整作业的 Vertex 完全限定标识符。保存此标识符以在后续步骤中使用。

`response.state`：自定义超参数调整作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获取自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
hpt_job_id = response.name
# The short numeric ID for the custom job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

### 获取超参数调整作业的信息

接下来，使用这个辅助函数 `get_hyperparameter_tuning_job`，它接受以下参数：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_hyperparameter_tuning_job` 方法，参数为：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

如果你还记得，你在调用 `create_hyperparameter_tuning_job` 方法时，在 `response.name` 字段中获得了超参数调整作业的 Vertex 完全限定标识符，并将其保存在变量 `hpt_job_id` 中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

等待调整完成

调整上述模型的超参数可能需要长达20分钟的时间。

一旦您的模型完成调整，您可以通过将`end_time`减去`start_time`来计算调整模型所需的实际时间。

对于您的模型，我们需要知道每次尝试保存模型的位置，该Python脚本将在您本地云存储桶中保存在`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

### 检讨研究结果

现在检讨试验的结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

### 最佳试验

现在看看哪个试验是最好的：

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

获取最佳模型

如果您使用了让服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

加载保存的模型

您的模型以TensorFlow SavedModel格式存储在云存储桶中。 现在从云存储桶加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载，您可以使用TF.Keras的`model.load_model()`方法，将保存模型的云存储路径（由`MODEL_DIR`指定）传递给它。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将使用`tf.keras.datasets`中的`load_data()`方法加载CIFAR10测试（留存）数据。 这将返回一个包含两个元素的元组数据集。 第一个元素是训练数据，第二个是测试数据。 每个元素也是一个包含两个元素的元组：图像数据和对应的标签。

您不需要训练数据，因此我们将其载入为 `(_,_)`。

在您运行数据进行评估之前，您需要对其进行预处理：

x_test:
1. 通过将每个像素除以255来归一化（重新缩放）像素数据。 这将用0到1之间的32位浮点数替换每个单字节整数像素。

y_test:
2. 标签当前是标量（稀疏）的。 如果您回顾`trainer/task.py`脚本中的`compile()`步骤，您会发现它是为稀疏标签进行编译的。 因此我们不需要进行其他操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估一下定制工作中的模型效果如何。

In [ ]:
model.evaluate(x_test, y_test)

清理工作

为了清理在本项目中使用的所有GCP资源，您可以[删除用于教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME